In [1]:
import lightgbm as lgb
import os
import pandas as pd
import numpy as np
import random
import gc
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score

np.random.seed(2019)
random.seed(2019)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 280)
pd.set_option('display.max_colwidth', 150)
data_path = '/data/workspace/kimi/tencent_ads/2020/dataset'

In [6]:
train_df =pd.read_pickle('train.pkl')
train_df.replace("\\N",-1,inplace=True)
train_df=train_df.astype(float,inplace=True)
train_df['age']  = train_df['age'] -1
train_df['gender']  = train_df['gender'] -1

In [14]:
final_train_x_df = train_df.drop(['active_days', 'click_times_total','age','user_id','max_clicked_industry_cnt','max_clicked_advertiser_cnt','max_clicked_industry_ratio','max_clicked_advertiser_ratio'], axis=1)
final_train_y_df = train_df['gender']
train_data_show_df =train_df.drop(['active_days', 'click_times_total','age','max_clicked_industry_cnt','clicked_industry','clicked_advertiser','max_clicked_advertiser_cnt','max_clicked_industry_ratio','max_clicked_advertiser_ratio'], axis=1)
print(train_data_show_df)
                                
#train_data = lgb.Dataset(final_train_x_df, label=final_train_y_df, feature_name=[   'max_clicked_industry', 'max_clicked_advertiser_id' ], categorical_feature=['max_clicked_industry','max_clicked_advertiser_id'])
train_data = lgb.Dataset(final_train_x_df, label=final_train_y_df, feature_name=[ 'active_days', 'click_times_total', 'max_clicked_industry', 'max_clicked_industry_cnt', 'clicked_industry', 'max_clicked_advertiser_id', 'max_clicked_advertiser_cnt', 'clicked_advertiser',  'max_clicked_industry_ratio', 'max_clicked_advertiser_ratio'], categorical_feature=['max_clicked_industry','max_clicked_advertiser_id'])


         user_id  max_clicked_industry  max_clicked_advertiser_id  gender
0            1.0                 326.0                      188.0     0.0
1            2.0                   6.0                    42272.0     0.0
2            3.0                 322.0                     8371.0     1.0
3            4.0                  -1.0                     8877.0     0.0
4            5.0                   6.0                     2862.0     0.0
5            6.0                  54.0                     7115.0     0.0
6            7.0                  54.0                     2421.0     1.0
7            8.0                   6.0                     6783.0     0.0
8            9.0                 329.0                    21035.0     0.0
9           10.0                   6.0                    14681.0     1.0
10          11.0                   6.0                    14681.0     1.0
11          12.0                   6.0                    21451.0     0.0
12          13.0                 319.0

In [20]:
params = {
    #'boosting_type': 'gbdt',
    'objective': 'softmax',
    'num_class':2,
    'metric': 'multi_error',
    'num_leaves': 31,
    'learning_rate': 0.1,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    'verbose': 0
}
print('Starting training...')
# train

gbm = lgb.train(params,
                train_data)
                #early_stopping_rounds=5)

print(gbm)

Starting training...


/home/ubuntu/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


In [21]:
y_pred = gbm.predict(train_df)

print(y_pred)

for i in range(len(y_pred)):
        max_value=max(y_pred[i])
        for j in range(len(y_pred[i])):
            if max_value==y_pred[i][j]:
                y_pred[i][j]=1
            else:
                y_pred[i][j]=0
print(y_pred)

[[2.86834409e-05 9.99971317e-01]
 [2.86834409e-05 9.99971317e-01]
 [2.86834409e-05 9.99971317e-01]
 ...
 [2.86834409e-05 9.99971317e-01]
 [2.86834409e-05 9.99971317e-01]
 [2.86834409e-05 9.99971317e-01]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [22]:
print(final_train_y_df.values)
from sklearn.preprocessing import  OneHotEncoder

before_one_hot =  final_train_y_df.values.reshape([-1,1])
print(before_one_hot)
enc = OneHotEncoder()
enc.fit(before_one_hot)

one_hoted_y  = enc.transform(before_one_hot).toarray()
print(one_hoted_y.shape)


[0. 0. 1. ... 1. 0. 1.]
[[0.]
 [0.]
 [1.]
 ...
 [1.]
 [0.]
 [1.]]
(900000, 2)


/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [23]:

classification_report(one_hoted_y, y_pred)
precision_score(one_hoted_y, y_pred,average='micro')

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.33043333333333336